In [ ]:
import torch
from torch import nn
from torch import tensor
from torch.utils.data import  DataLoader
from torch import optim
from torch.optim import lr_scheduler
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import argparse
import os
pd.set_option('display.max_columns', None)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
def model_parameters_init(model):
    '''
    kaiming init
    '''
    for p in model.parameters():
        if len(p.shape) >= 2:
            nn.init.kaiming_normal_(p)
    return model

In [ ]:
class NetWork(nn.Module):
    '''
    只输入一个aqi，其他置0
    '''
    def __init__(self):
        super(NetWork,self).__init__()
        self.lstm = nn.LSTM(24, 144, 1, batch_first= True)
        self.linear_1 = nn.Linear(144, 72)
        self.linear_2 = nn.Linear(72, 1)
        self.relu = nn.PReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)
        self.bn = nn.BatchNorm1d(96)
        
    def forward(self, x):
        x = self.bn(x)
        x,(h_1,c_1) = self.lstm(x)
        x = x[:,24:,:]
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear_1(x)
        x = self.relu(x)
        x = self.linear_2(x)
        x = x.squeeze(2)
        return x

In [ ]:
class ToTensor(object):
    '''
    transform sample to tensor
    '''
    def __call__(self, sample):
        sample['x'] = torch.from_numpy(sample['x']).float()
        sample['y'] = torch.from_numpy(sample['y']).float()
        return sample

In [ ]:
class AqiDataset():
    '''
    用于获取aqi训练与测试数据
    '''
    def __init__(self, data, transforms= None):
        '''
        data: samples of ndarray from csv
        '''
        self.data = data.copy()
        #前24气象要素用不到，只取aqi
        #后面71个取不到
        self.size = len(data) - 71 - 24
        self.transforms = transforms
    
    def __call__(self):
        print('使用__getitem__(idx)获取数据')
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        '''
        定义了__getitem__魔法函数，该类就可以下标操作了：[]
        '''
        #历史24时刻数据
        history = self.data[idx : idx + 24].copy()
        #未来72时刻数据（不包括AQI）
        future = self.data[idx + 24 : idx + 24 + 72].copy()
        future[:,0] = 0
        #合并
        h_f = np.r_[history, future]
        #取各个时刻的实测值
        lables = self.data[idx + 24 : idx + 24 + 72, 0].copy()
        #进行必要的变换
        sample = {'x':h_f, 'y':lables}
        sample = self.transforms(sample)
        return sample

In [ ]:
def train(args, train_loader, valid_loader, model, criterion, optimizer, scheduler, device):
    #save model or not
    if args.save_model:
        if not os.path.exists(args.save_directory):
            os.makedirs(args.save_directory)
    
    epochs = args.epochs
    train_losses = []
    valid_losses = []
    for epoch_id in range(epochs):
        #monitor training loss
        train_loss = 0.0
        valid_loss = 0.0
        model.train()
        ######################
        #training the model#
        ######################
        train_batch_cnt = 0
        for batch_idx, batch in enumerate(train_loader):
            train_batch_cnt += 1
            x = batch['x']
            y = batch['y']
            
            # groundtruth
            x = x.to(device)
            y = y.to(device)
            
            #clear the gradients of all optimized variables
            optimizer.zero_grad()
            
            #get out_puts
            pred_y = model(x)
            
            #get loss
            loss = criterion(y, pred_y)
            train_loss += loss.item()
            
            #do bp
            loss.backward()
            optimizer.step()
            
            #show log info
            if batch_idx % args.log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]  MSELoss: {:.6f}'.format(
                        epoch_id,
                        batch_idx * len(x),
                        len(train_loader.dataset),
                        100. * batch_idx / len(train_loader),
                        loss.item()
                        )
                      )
        #记录train_loss
        train_loss /= train_batch_cnt
        train_losses.append(train_loss)
            
        ######################
        # validate the model #
        ######################
        valid_loss = 0.0
        #change model mode to eval ,not use BN/Dropout
        model.eval()
        with torch.no_grad():
            valid_batch_cnt = 0

            for valid_batch_idx, batch in enumerate(valid_loader):
                valid_batch_cnt += 1
                x = batch['x']
                y = batch['y']

                x = x.to(device)
                y = y.to(device)

                pred_y = model(x)
                
                valid_loss_batch = criterion(y, pred_y)
                valid_loss += valid_loss_batch.item()

            valid_loss /= valid_batch_cnt * 1.0
            #记录valid_loss
            valid_losses.append(valid_loss)
            print('Valid: MSELoss: {:.6f}'.format(valid_loss))
        #学习率衰减
        scheduler.step()
        print('===========================================================')
        #save model
        if args.save_model and epoch_id % 10 == 0:
            saved_model_name = os.path.join(args.save_directory, 'epoch' + '_' + str(epoch_id) + '.pt')
            torch.save(model.state_dict(), saved_model_name)
    return train_losses, valid_losses

In [ ]:
def test(args, valid_loader, model, criterion, device):
    path_model = os.path.join(args.save_directory, 'epoch' + '_' + str(args.number) + '.pt')
    model.load_state_dict(torch.load(path_model))
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        valid_batch_cnt = 0
        valid_loss = 0
        mb_y = 0
        mb_pred_y = 0
        for valid_batch_idx, batch in enumerate(valid_loader):
            valid_batch_cnt += 1
            x = batch['x']
            y = batch['y']
        
            x = x.to(device)
            y = y.to(device)
            pred_y = model(x)
            
            mb_y += y.mean()
            mb_pred_y += pred_y.mean()
            
            valid_loss_batch = criterion(y, pred_y)
            valid_loss += valid_loss_batch.item()

        valid_loss /= valid_batch_cnt * 1.0
        print('Valid: MSELoss: {:.6f}'.format(valid_loss))
        print('MB_Y: {:.2f}, MB_PredY: {:.2f}'.format(mb_y.cpu().numpy(), mb_pred_y.cpu().numpy()))

In [ ]:
def predict(args, val_set, model, device):
    path_model = os.path.join(args.save_directory, 'epoch' + '_' + str(args.number) + '.pt')
    model.load_state_dict(torch.load(path_model))
    model = model.to(device)
    model.eval()
    data = val_set[args.idx]
    with torch.no_grad():
        x = data['x'].to(device).unsqueeze(0)
        y = data['y'].numpy()
        pred_y = model(x)
        pred_y = pred_y.cpu().numpy()
        plt.figure(0,(8,6))
        plt.plot(range(len(y)),y,'b-o')
        plt.plot(range(len(y)),pred_y[0],'r-o')
        plt.xlabel('Following 72 Hours')
        plt.ylabel('AQI')
        plt.legend(['obs','predict'])
        plt.show()

In [ ]:
def main(args):
    #设置随机种子
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    #设置CPU/GPU
    use_cuda = args.cuda and torch.cuda.is_available()
    device = torch.device('cuda' if use_cuda else 'cpu')
    ###############################################################################################################
    print('===> Loading Datasets')
    #读数据
    df = pd.read_csv('data/aqi_upsample.csv',index_col= 0)
    data = df.values
    data[:,4] /= 10000
    #划分样本集
    trsf = transforms.Compose([ToTensor()])
    train_set = AqiDataset(data[:18000], trsf)
    test_set = AqiDataset(data[18000:27000], trsf)
    val_set = AqiDataset(data[27000:], trsf)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=args.batch_size, shuffle=False, num_workers= 1, pin_memory= True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=args.test_batch_size, num_workers= 1, pin_memory= True)
    predict_loader = torch.utils.data.DataLoader(val_set, batch_size=args.predict_batch_size, num_workers= 1, pin_memory= False)
    ###############################################################################################################
    print('===> Building Model')
    print('===> runing on {}'.format(device))
    ###############################################################################################################
    print('===> init model')
    model = NetWork()
    ###############################################################################################################
    model.to(device)
#     criterion = nn.MSELoss()
    criterion = nn.SmoothL1Loss()
#     optimizer = optim.Adam(model.parameters(), lr= args.lr)
    optimizer = optim.SGD(model.parameters(), lr = args.lr, momentum= args.momentum)
    #学习率衰减
    scheduler = optim.lr_scheduler.StepLR(optimizer, 1 , 0.85)
    ###############################################################################################################
    if args.phase == 'Train' or args.phase == 'train':
        print('===> Start Training')
        train_losses, valid_losses = train(args, train_loader, test_loader, model, criterion, optimizer, scheduler, device)
        print('===> Done!')
        return train_losses, valid_losses
        
    elif args.phase == 'Test' or args.phase == 'test':
        print('===> Test')
        test(args, test_loader, model, criterion, device)
        print('===> Done!')
        return None, None
    elif args.phase == 'Finetune' or args.phase == 'finetune':
        print('===> Finetune')
        path_model = os.path.join(args.save_directory, 'epoch' + '_' + str(args.number) + '.pt')
        model.load_state_dict(torch.load(path_model))
        model = model.to(device)
        train_losses, valid_losses = train(args, train_loader, valid_loader, model, criterion, optimizer, scheduler, device)
        print('===> Done!')
        return train_losses, valid_losses
        
    elif args.phase == 'Predict' or args.phase == 'predict':
        print('===> Predict')
        predict(args, val_set, model, device)
        print('===> Done!')
        return None, None

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Detector')
    parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 256)')
    parser.add_argument('--test_batch_size', type=int, default=256, metavar='N',
                        help='input batch size for testing (default: 256)')
    parser.add_argument('--predict_batch_size', type=int, default=1, metavar='N',
                        help='input batch size for predict (default: 1)')
    parser.add_argument('--epochs', type=int, default=50, metavar='N',
                        help='number of epochs to train (default: 100)')
    parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                        help='learning rate (default: 0.001)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--cuda', action='store_true', default=False,
                        help='enables CUDA training')
    parser.add_argument('--seed', type=int, default=10, metavar='S',
                        help='random seed (default: 10)')
    parser.add_argument('--log_interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save_model', action='store_true', default=False,
                        help='save the current Model')
    parser.add_argument('--save_directory', type=str, default='trained_models',
                        help='learnt models are saving here')
    parser.add_argument('--phase', type=str, default='Train',   # Train/train, Predict/predict, Finetune/finetune
                        help='training, predicting or finetuning')
    parser.add_argument('--number', type=int, default=0,
                        help='which model to use')
    parser.add_argument('--idx', type=int, default=0,
                        help='which sample to predict')
    args = parser.parse_args(['--batch_size=64',
                              '--test_batch_size=2048',
                              '--predict_batch_size=1',
                              '--epochs=101',
                              '--lr=0.01',
                              '--momentum=0.5',
                              '--cuda',
                              '--seed=1',
                              '--log_interval=50',
                              '--save_model',
                              '--save_directory=trained_models_2',
                              '--number=100',
                              '--idx=1850',
                              '--phase=train'])
    ##############################################################################################################
    start = time.time()
    train_losses, valid_losses = main(args)
    end = time.time()
    print('耗时：{}s'.format(end - start))
    torch.cuda.empty_cache()

In [ ]:
#读数据
df = pd.read_csv('data/aqi_upsample.csv',index_col= 0)
data = df.values
data[:,4] /= 10000
#划分样本集
trsf = transforms.Compose([ToTensor()])
train_set = AqiDataset(data[:18000], trsf)
test_set = AqiDataset(data[18000:27000], trsf)
val_set = AqiDataset(data[27000:], trsf)